# Disclaimer
En este script se realizará la fase de modelado de la problemática, teniendo en cuenta tanto diferentes posibles modelos, como técnicas (ensamble, redes neuronales, etc.)

In [76]:
# Imports generales
import pandas as pd
import io
import numpy as np
import statistics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import cluster
import sklearn.metrics
from sklearn.cluster import DBSCAN
from sklearn.neighbors import kneighbors_graph
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score

from sklearn.metrics import davies_bouldin_score

In [77]:
path = 'C:/Users/Usuario/Documents/TFM/'

In [78]:
famd = pd.read_csv( path + 'FAMD_municipios.csv', sep = ';')


base = pd.read_excel(path + 'Base_modelado.xlsx')
mpios = pd.read_excel( path + 'DNP mpios2025 base mpios.xlsx', sheet_name = 'Municipios')



In [79]:
base['MUNICIPIO_ANIO'] = base['MPIO'].astype(str) + '_' + base['ANIO_DES'].astype(str)

In [80]:
mpios['Tipología_2025'] = mpios['Tipología_2025'].astype(str)

mpios = mpios[['MPIO', 'CAT_ruralidad 2023']]

In [81]:
base['ANIO_MPIO'] = base['MPIO'].astype(str) + '_' + base['ANIO_DES'].astype(str)

In [82]:
base = base.merge(mpios, on = 'MPIO', how = 'left')

In [83]:
base['CAT_ruralidad 2023'] = base['CAT_ruralidad 2023'].fillna('SIN_INFO')


In [84]:
reemplazos = { 
              'Aparecio vivo': 1,
              'Aparecio muerto': 1,
              'Desaparecido': 0}

base['ESTADO'] = base['ESTADO'].replace(reemplazos)

reemplazos = { 
              'basica_secundaria': 'Secundaria',
              'secundaria': 'Secundaria',
              'primaria': 'Primaria',
              'inicial y preescolar': 'Primaria',
              'profesional': 'Avanzada',
              'Universitario': 'Avanzada',
              'master o especializacion': 'Avanzada',
              'doctorado': 'Avanzada',
              'Sin escolaridad': 'Sin escolaridad',
              'SIN_INFO': 'SIN_INFO'}

base['ESCOLARIDAD'] = base['ESCOLARIDAD'].replace(reemplazos)


reemplazos = { 
              'Aparecio vivo': 'Aparecio',
              'Aparecio muerto': 'Aparecio',
              'Desaparecido': 'Desaparecido'}

base['ESTADO'] = base['ESTADO'].replace(reemplazos)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_17928\1551899384.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  base['ESTADO'] = base['ESTADO'].replace(reemplazos)


In [85]:
print('La distribución de nuestra variable objetivo es:')
base.groupby('ESTADO')['ID'].count()

La distribución de nuestra variable objetivo es:


ESTADO
0    123348
1     67558
Name: ID, dtype: int64

In [86]:
columnas_a_quitar = [
    'Unnamed: 0',
    'ENTIDAD',
    'EDAD_GRUPO',
    'EDAD_QUINQ',
    'PRT_GRUP',
    'FECHA_DES',
    'DIA_DES',
    'PAIS_DES',
    'DEPTO',
    'DEPTO_DES',
    'MPIO_DES',
    'ANIO_MPIO',
    'ZONA_DES',
    'MES_DES',
    'CLASIFICACION'
]

Base = base.drop(columns=columnas_a_quitar)


In [87]:
# print(Base['ESTADO'].unique())
# # print('__________________________________________')
# # print(Base['CLASIFICACION'].unique())
# print('__________________________________________') 
# print(Base['SEXO'].unique())
# print('__________________________________________') 
# print(Base['CICLO_VIT'].unique())
# print('__________________________________________') 
# print(Base['EST_CIVIL'].unique())
# print('__________________________________________') 
# print(Base['ESCOLARIDAD'].unique())
# print('__________________________________________') 
# # print(Base['ANIO_DES'].unique())
# # print('__________________________________________') 
# print(Base['MES_DES'].unique())
# print('__________________________________________') 
# print(Base['CAT_ruralidad 2023'].unique())
# print('__________________________________________') 


Base.set_index('ID', inplace = True)

## Transformación MPIOS

In [88]:
Base

,ESTADO,SEXO,CICLO_VIT,EST_CIVIL,ESCOLARIDAD,ANIO_DES,MPIO,MUNICIPIO_ANIO,CAT_ruralidad 2023
ID,,,,,,,,,
1,0,Hombre,Juventud,Soltero,Secundaria,2006,11001,11001_2006,Ciudades y aglomeraciones
2,0,Hombre,Juventud,Soltero,Secundaria,2006,11001,11001_2006,Ciudades y aglomeraciones
3,0,Mujer,Adolescencia,Soltero,Secundaria,2006,11001,11001_2006,Ciudades y aglomeraciones
4,1,Mujer,Adolescencia,Soltero,Avanzada,2006,11001,11001_2006,Ciudades y aglomeraciones
5,1,Hombre,Adulto_mayor,Union_libre,Primaria,2006,11001,11001_2006,Ciudades y aglomeraciones
...,...,...,...,...,...,...,...,...,...
193048,0,Hombre,Juventud,Soltero,SIN_INFO,2007,95001,95001_2007,Rural
193049,0,Hombre,Juventud,Soltero,Primaria,2011,5250,5250_2011,Intermedio
193050,0,Hombre,Juventud,SIN_INFO,SIN_INFO,2001,68235,68235_2001,Rural disperso


In [89]:
a = Base.groupby('MUNICIPIO_ANIO')['ESTADO'].count()


import numpy as np
import pandas as pd

# 'a' es una Serie de pandas: índice = MPIO, valores = conteo de ESTADO
# Por ejemplo:
# a = Base.groupby('MPIO')['ESTADO'].count()

# Convertimos a un arreglo NumPy
counts = a.values

# Definimos los percentiles que queremos calcular
percentil_vals = [0, 25, 50, 75, 100]

# Calculamos los percentiles
percentiles = np.percentile(counts, percentil_vals)

# Convertimos a DataFrame legible
df_percentiles = pd.DataFrame({
    'Percentil': percentil_vals,
    'Conteo': percentiles.astype(int)
})
print(df_percentiles)


   Percentil  Conteo
0          0       1
1         25       1
2         50       2
3         75       5
4        100    3606


In [91]:
import pandas as pd
import numpy as np

# Supongamos que tienes
# Base: tu DataFrame original con columnas 'MPIO' y 'ESTADO'
# a: Serie con el conteo de desapariciones por municipio
a = Base.groupby('MUNICIPIO_ANIO')['ESTADO'].count()

# Opcional: revisa cuántos hay por municipio
counts = a

# Umbral de desapariciones
threshold = 20

# Identifica aquellos municipios con conteo menor que el umbral
mpios_menores = counts[counts < threshold].index

# Ahora creas una nueva columna, por ej. 'MPIO_reclasificado'
Base['MPIO_ANIO_reclasificado'] = np.where(
    Base['MUNICIPIO_ANIO'].isin(mpios_menores),
    'Otros',
    Base['MUNICIPIO_ANIO']
)


## Inicio Embedding

In [93]:
import pandas as pd

# Convertir MPIO a entero consecutivo
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
Base["MPIO_ANIO_ID"] = encoder.fit_transform(Base["MPIO_ANIO_reclasificado"])  # ej: Bogotá = 0, Medellín = 1, etc.

# Guardamos cuántos municipios únicos hay
num_mpios = Base["MPIO_ANIO_ID"].nunique()
Base

,ESTADO,SEXO,CICLO_VIT,EST_CIVIL,ESCOLARIDAD,ANIO_DES,MPIO,MUNICIPIO_ANIO,CAT_ruralidad 2023,MPIO_ANIO_reclasificado,MPIO_ANIO_ID
ID,,,,,,,,,,,
1,0,Hombre,Juventud,Soltero,Secundaria,2006,11001,11001_2006,Ciudades y aglomeraciones,11001_2006,23
2,0,Hombre,Juventud,Soltero,Secundaria,2006,11001,11001_2006,Ciudades y aglomeraciones,11001_2006,23
3,0,Mujer,Adolescencia,Soltero,Secundaria,2006,11001,11001_2006,Ciudades y aglomeraciones,11001_2006,23
4,1,Mujer,Adolescencia,Soltero,Avanzada,2006,11001,11001_2006,Ciudades y aglomeraciones,11001_2006,23
5,1,Hombre,Adulto_mayor,Union_libre,Primaria,2006,11001,11001_2006,Ciudades y aglomeraciones,11001_2006,23
...,...,...,...,...,...,...,...,...,...,...,...
193048,0,Hombre,Juventud,Soltero,SIN_INFO,2007,95001,95001_2007,Rural,95001_2007,1166
193049,0,Hombre,Juventud,Soltero,Primaria,2011,5250,5250_2011,Intermedio,Otros,1191
193050,0,Hombre,Juventud,SIN_INFO,SIN_INFO,2001,68235,68235_2001,Rural disperso,Otros,1191


In [94]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense

# Crear input de municipio
input_mpio = Input(shape=(1,), name="mpio_input")
embedding_dim = 15  # puedes probar con 4, 8, 16...

embedding = Embedding(input_dim=num_mpios, output_dim=embedding_dim, name="mpio_embedding")(input_mpio)
flat = Flatten()(embedding)

# Capa de salida binaria (apareció o no)
output = Dense(1, activation='sigmoid')(flat)

# Modelo
model = Model(inputs=input_mpio, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Entrenar el Embedding

In [95]:
X_mpio = Base['MPIO_ANIO_ID'].values

y = Base['ESTADO'].values

model.fit(X_mpio,y, epochs = 15, batch_size= 64)

Epoch 1/15
2983/2983 ━━━━━━━━━━━━━━━━━━━━ 2s 481us/step - accuracy: 0.7135 - loss: 0.5776
Epoch 2/15
2983/2983 ━━━━━━━━━━━━━━━━━━━━ 1s 479us/step - accuracy: 0.7320 - loss: 0.5346
Epoch 3/15
2983/2983 ━━━━━━━━━━━━━━━━━━━━ 1s 475us/step - accuracy: 0.7322 - loss: 0.5316
Epoch 4/15
2983/2983 ━━━━━━━━━━━━━━━━━━━━ 1s 473us/step - accuracy: 0.7357 - loss: 0.5290
Epoch 5/15
2983/2983 ━━━━━━━━━━━━━━━━━━━━ 1s 481us/step - accuracy: 0.7340 - loss: 0.5299
Epoch 6/15
2983/2983 ━━━━━━━━━━━━━━━━━━━━ 1s 484us/step - accuracy: 0.7344 - loss: 0.5294
Epoch 7/15
2983/2983 ━━━━━━━━━━━━━━━━━━━━ 1s 475us/step - accuracy: 0.7349 - loss: 0.5294
Epoch 8/15
2983/2983 ━━━━━━━━━━━━━━━━━━━━ 1s 469us/step - accuracy: 0.7339 - loss: 0.5292
Epoch 9/15
2983/2983 ━━━━━━━━━━━━━━━━━━━━ 1s 482us/step - accuracy: 0.7344 - loss: 0.5309
Epoch 10/15
2983/2983 ━━━━━━━━━━━━━━━━━━━━ 1s 476us/step - accuracy: 0.7339 - loss: 0.5298
Epoch 11/15
2983/2983 ━━━━━━━━━━━━━━━━━━━━ 1s 471us/step - accuracy: 0.7346 - loss: 0.5293
Epoch 12

In [96]:
# 1. Extract the embedding matrix
embedding_matrix = model.get_layer('mpio_embedding').get_weights()[0]

# 2. Create column names for each embedding dimension
column_names = [f"MPIO_ANIO_emb_{i}" for i in range(embedding_matrix.shape[1])]

# 3. Build the DataFrame
embedding_df = pd.DataFrame(embedding_matrix, columns=column_names)

# 4. Add an identifier column for each municipio
embedding_df["MPIO_ANIO_ID"] = range(embedding_matrix.shape[0])


In [97]:
embedding_df

,MPIO_ANIO_emb_0,MPIO_ANIO_emb_1,MPIO_ANIO_emb_2,MPIO_ANIO_emb_3,MPIO_ANIO_emb_4,MPIO_ANIO_emb_5,MPIO_ANIO_emb_6,MPIO_ANIO_emb_7,MPIO_ANIO_emb_8,MPIO_ANIO_emb_9,MPIO_ANIO_emb_10,MPIO_ANIO_emb_11,MPIO_ANIO_emb_12,MPIO_ANIO_emb_13,MPIO_ANIO_emb_14,MPIO_ANIO_ID
0,-1.888538,2.172012,2.265386,-2.192933,2.292042,2.254899,2.364782,-2.334738,-1.734660,1.848574,1.977468,2.208464,1.946184,2.113732,-2.216842,0
1,-0.376237,0.385143,0.363121,-0.423721,0.419459,0.406479,0.368966,-0.371151,-0.344940,0.395034,0.426943,0.356730,0.437869,0.379336,-0.403834,1
2,0.039618,0.023950,0.012798,0.029846,0.014648,0.000247,0.002917,0.010928,0.036929,0.021274,0.027683,0.015898,0.027165,-0.033565,0.043493,2
3,-0.516563,0.471578,0.480380,-0.477308,0.544374,0.520041,0.531000,-0.496926,-0.457740,0.534856,0.497177,0.486858,0.495656,0.528146,-0.503145,3
4,-0.416819,0.445009,0.405680,-0.452477,0.515986,0.429363,0.478471,-0.439283,-0.481234,0.465419,0.483056,0.407647,0.450878,0.411406,-0.463641,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,-0.492949,0.494227,0.421888,-0.422217,0.463206,0.411454,0.443941,-0.485451,-0.466207,0.499114,0.491813,0.495187,0.504085,0.428577,-0.503858,1187
1188,-1.785931,1.972105,2.022326,-2.019043,2.065382,2.013771,2.074372,-2.169911,-1.650449,1.762242,1.876520,2.029964,1.735847,1.917961,-2.023400,1188
1189,-0.412999,0.346011,0.389579,-0.341026,0.356621,0.334587,0.324800,-0.365062,-0.360445,0.386917,0.323738,0.322820,0.369378,0.331014,-0.395291,1189
1190,-0.751917,0.756581,0.808824,-0.786517,0.771921,0.738368,0.813665,-0.767207,-0.735574,0.770913,0.822672,0.749816,0.750479,0.744001,-0.737547,1190


In [98]:
Base_final = pd.merge(Base, embedding_df, on = 'MPIO_ANIO_ID', how = 'left')

In [100]:
Base_final

,ESTADO,SEXO,CICLO_VIT,EST_CIVIL,ESCOLARIDAD,ANIO_DES,MPIO,MUNICIPIO_ANIO,CAT_ruralidad 2023,MPIO_ANIO_reclasificado,...,MPIO_ANIO_emb_5,MPIO_ANIO_emb_6,MPIO_ANIO_emb_7,MPIO_ANIO_emb_8,MPIO_ANIO_emb_9,MPIO_ANIO_emb_10,MPIO_ANIO_emb_11,MPIO_ANIO_emb_12,MPIO_ANIO_emb_13,MPIO_ANIO_emb_14
0,0,Hombre,Juventud,Soltero,Secundaria,2006,11001,11001_2006,Ciudades y aglomeraciones,11001_2006,...,0.182609,0.232895,-0.221277,-0.220405,0.193655,0.217458,0.185064,0.194149,0.199805,-0.155760
1,0,Hombre,Juventud,Soltero,Secundaria,2006,11001,11001_2006,Ciudades y aglomeraciones,11001_2006,...,0.182609,0.232895,-0.221277,-0.220405,0.193655,0.217458,0.185064,0.194149,0.199805,-0.155760
2,0,Mujer,Adolescencia,Soltero,Secundaria,2006,11001,11001_2006,Ciudades y aglomeraciones,11001_2006,...,0.182609,0.232895,-0.221277,-0.220405,0.193655,0.217458,0.185064,0.194149,0.199805,-0.155760
3,1,Mujer,Adolescencia,Soltero,Avanzada,2006,11001,11001_2006,Ciudades y aglomeraciones,11001_2006,...,0.182609,0.232895,-0.221277,-0.220405,0.193655,0.217458,0.185064,0.194149,0.199805,-0.155760
4,1,Hombre,Adulto_mayor,Union_libre,Primaria,2006,11001,11001_2006,Ciudades y aglomeraciones,11001_2006,...,0.182609,0.232895,-0.221277,-0.220405,0.193655,0.217458,0.185064,0.194149,0.199805,-0.155760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190901,0,Hombre,Juventud,Soltero,SIN_INFO,2007,95001,95001_2007,Rural,95001_2007,...,0.417523,0.328758,-0.369017,-0.406644,0.375692,0.425469,0.357956,0.340475,0.403404,-0.396239
190902,0,Hombre,Juventud,Soltero,Primaria,2011,5250,5250_2011,Intermedio,Otros,...,0.292268,0.288021,-0.310609,-0.297561,0.296113,0.281529,0.317248,0.303181,0.264986,-0.331226
190903,0,Hombre,Juventud,SIN_INFO,SIN_INFO,2001,68235,68235_2001,Rural disperso,Otros,...,0.292268,0.288021,-0.310609,-0.297561,0.296113,0.281529,0.317248,0.303181,0.264986,-0.331226
190904,0,Mujer,Juventud,SIN_INFO,SIN_INFO,1984,68235,68235_1984,Rural disperso,Otros,...,0.292268,0.288021,-0.310609,-0.297561,0.296113,0.281529,0.317248,0.303181,0.264986,-0.331226


In [101]:
Base_final.to_excel(path + 'Embeding_MPIO_ANIO_ajustados_otros.xlsx')